# CSV Generator

## Intención de este notebook

Generar el CSV con la información final de los vuelos, que será utilizando en Neo4j para resolver las consultas propuestas por la cátedra.

## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians

## Procesamiento de los datos

### Carga del archivo pkl

In [28]:
info_vuelos = pd.read_pickle('pkl/info_vuelos.pkl')
info_vuelos = info_vuelos.drop(columns=['vuelo', 'terminal_salida', 'puerta_salida', 'terminal_llegada', 'puerta_llegada'])
info_vuelos.sample(5)

,aerolinea,origen,destino,fecha_salida,horario_salida,fecha_llegada,horario_llegada
134505,Etihad,Abu Dabi,Pekín-Capital,"[sabado, 16, marzo]",21:18,"[domingo, 17, marzo]",23:29
20182,American,Chicago O'Hare,Toledo,"[viernes, 8, marzo]",16:40,"[viernes, 8, marzo]",18:38
52483,Aerolíneas Argentinas,Roma Fiumicino,Buenos Aires Ezeiza,"[jueves, 14, marzo]",20:43,"[viernes, 15, marzo]",00:38
65095,Alitalia,Clermont-Ferrand,París Charles de Gaulle,"[miercoles, 13, marzo]",19:54,"[miercoles, 13, marzo]",06:09
46866,AirAsia,Archipiélago Langkawi,Kuala Lumpur,"[martes, 12, marzo]",15:30,"[martes, 12, marzo]",18:35


In [29]:
info_vuelos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 344124 entries, 0 to 196239
Data columns (total 7 columns):
aerolinea          344124 non-null object
origen             344124 non-null object
destino            344124 non-null object
fecha_salida       344124 non-null object
horario_salida     344124 non-null object
fecha_llegada      344124 non-null object
horario_llegada    344124 non-null object
dtypes: object(7)
memory usage: 21.0+ MB


### Eliminación de espacios

In [30]:
info_vuelos['aerolinea'] = info_vuelos['aerolinea'].apply(lambda x: x.strip())
info_vuelos['origen'] = info_vuelos['origen'].apply(lambda x: x.strip())
info_vuelos['destino'] = info_vuelos['destino'].apply(lambda x: x.strip())
info_vuelos.sample(5)

,aerolinea,origen,destino,fecha_salida,horario_salida,fecha_llegada,horario_llegada
144581,Gol,Aracaju,São Paulo Guarulhos,"[viernes, 15, marzo]",06:50,"[viernes, 15, marzo]",10:49
10424,American,Florence,Charlotte,"[domingo, 10, marzo]",08:24,"[domingo, 10, marzo]",10:44
14332,American,Londres Heathrow,Bangalore,"[sabado, 9, marzo]",13:55,"[domingo, 10, marzo]",05:00
15631,American,Kansas City,Charlotte,"[viernes, 8, marzo]",19:00,"[viernes, 8, marzo]",20:25
71332,British Airways,Amán,Londres Heathrow,"[lunes, 11, marzo]",08:01,"[lunes, 11, marzo]",09:52


### Conversión de fechas y horarios

In [31]:
# Obtengo el numero del mes
def get_month_number(month):
    months = {'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04', 'mayo': '05', 'junio': '06',\
              'julio': '07', 'agosto': '08', 'septiembre': '09]', 'octubre': 10, 'noviembre': 11, 'diciembre': 12}
    
    return months[month]

# Obtengo la fecha en str (YYYY-MM-DD)
def get_date(array):
    day = str(array[1])
    if len(day) == 1:
        day = '0' + day
    month = get_month_number(array[2])
    date = '2019' + '-' + month + '-' + day
    
    return date

# Actualizo los campos correspondientes
info_vuelos['fecha_salida'] = info_vuelos['fecha_salida'].apply(lambda x: get_date(x))
info_vuelos['fecha_llegada'] = info_vuelos['fecha_llegada'].apply(lambda x: get_date(x))

info_vuelos.sample(5)

,aerolinea,origen,destino,fecha_salida,horario_salida,fecha_llegada,horario_llegada
35004,Azul,São Paulo Viracopos,Belo Horizonte Confins,2019-03-09,20:55,2019-03-09,22:10
54153,Alaska,Juneau,Anchorage,2019-03-10,15:35,2019-03-10,17:40
75182,British Airways,Mánchester,Billund,2019-03-11,08:45,2019-03-11,15:20
40396,Air France,Indianápolis,Atlanta,2019-03-11,08:00,2019-03-11,09:10
95624,China Southern,Chongqing,Cantón,2019-03-12,17:00,2019-03-12,18:50


### Duración del vuelo (en minutos)

In [34]:
def get_duracion(departure_date, departure_hour, arrival_date, arrival_hour):
    dep_hour = int(departure_hour.replace(':', ''))
    arr_hour = int(arrival_hour.replace(':', ''))
    diff = arr_hour - dep_hour
    return diff

# TODO: completar esto
get_duracion('', '20:55', '', '22:10')

35

### Carga del archivo aeropuertos.csv para obtener precio del vuelo

In [36]:
aeropuertos = pd.read_csv('../csv/aeropuertos.csv', index_col=False, names=['codigo', 'aeropuerto', 'capital', 'pais', 'latitud', 'longitud', 'numero1', 'numero2', 'continente'])
aeropuertos.head(2)

,codigo,aeropuerto,capital,pais,latitud,longitud,numero1,numero2,continente
0,GKA,Goroka Airport,Goroka,Papua New Guinea,-6.08169,145.392,5282.0,10.00,Pacific/Port_Moresby
1,MAG,Madang Airport,Madang,Papua New Guinea,-5.20708,145.789,20.0,10.00,Pacific/Port_Moresby


### Códigos de los aeropuertos origen y destino 

In [37]:
a = info_vuelos.merge(aeropuertos[['codigo', 'capital']], left_on='origen', right_on='capital', how='left')
a.dropna(subset=['codigo'], inplace=True)

a = a[['aerolinea', 'origen', 'codigo', 'destino', 'fecha_salida', 'horario_salida', 'fecha_llegada', 'horario_llegada']]
a.columns = ['aerolinea', 'origen', 'cod_origen', 'destino', 'fecha_salida', 'horario_salida', 'fecha_llegada', 'horario_llegada']

a = a.merge(aeropuertos[['codigo', 'capital']], left_on='destino', right_on='capital', how='left')
a.dropna(subset=['codigo'], inplace=True)

a = a[['aerolinea', 'origen', 'cod_origen', 'destino', 'codigo', 'fecha_salida', 'horario_salida', 'fecha_llegada', 'horario_llegada']]
a.columns = ['aerolinea', 'origen', 'cod_origen', 'destino', 'cod_destino', 'fecha_salida', 'horario_salida', 'fecha_llegada', 'horario_llegada']

In [38]:
a.sample(5)

,aerolinea,origen,cod_origen,destino,cod_destino,fecha_salida,horario_salida,fecha_llegada,horario_llegada
224618,Delta,Columbia,CUB,Atlanta,PDK,2019-03-15,15:00,2019-03-15,20:05
239090,American,Charlotte,CLT,Wilmington,ILM,2019-03-10,19:42,2019-03-10,20:29
382552,Delta,Atlanta,PDK,Detroit,DTW,2019-03-15,18:50,2019-03-15,23:35
479648,Shandong,Xiamen,XMN,Jinan,TNA,2019-03-18,16:05,2019-03-18,19:35
89531,Air Canada,London,STN,Ottawa,YOW,2019-03-10,21:50,2019-03-10,05:30


### Funciones para calcular distancia y precio

In [39]:
aeropuertos = pd.read_csv('../csv/aeropuertos.csv', index_col=False, names=['codigo', 'aeropuerto',\
'capital', 'pais', 'latitud', 'longitud', 'numero1', 'numero2', 'continente'])

def get_coordenadas_aeropuerto(cod_aeropuerto):
    info_aeropuerto = aeropuertos[aeropuertos.codigo == cod_aeropuerto].head(1)

    return float(info_aeropuerto.latitud.item()), float(info_aeropuerto.longitud.item())


def calcular_distancia(aeropuerto_origen, aeropuerto_destino):
    aeropuerto_origen_latitud, aeropuerto_origen_longitud = get_coordenadas_aeropuerto(aeropuerto_origen)
    aeropuerto_destino_latitud, aeropuerto_destino_longitud = get_coordenadas_aeropuerto(aeropuerto_destino)
    
    # Approximated radius of earth in km
    R = 6373.0
    
    dlon = aeropuerto_destino_longitud - aeropuerto_origen_longitud
    dlat = aeropuerto_destino_latitud - aeropuerto_origen_latitud
    
    a = sin(dlat / 2)**2 + cos(aeropuerto_origen_latitud) * cos(aeropuerto_destino_latitud) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance


def calcular_precio(distancia_recorrida):
    # Precio por kilometro recorrido para un pasajero
    precio_dolares_por_km = 0.067
    precio = distancia_recorrida * precio_dolares_por_km
    iva = 0.21 * precio
    return precio + iva

### Precio del vuelo (proporcional a la distancia entre los aeropuertos)

In [43]:
a['precio'] = a.apply(lambda row: calcular_precio(calcular_distancia(row.cod_origen, row.cod_destino)), axis=1)
a.sample(5)

,aerolinea,origen,cod_origen,destino,cod_destino,fecha_salida,horario_salida,fecha_llegada,horario_llegada,precio
262522,American,San Francisco,SFO,Seattle,BFI,2019-03-07,01:50,2019-03-07,05:40,1369.238074
380094,Delta,Atlanta,FFC,Cincinnati,CVG,2019-03-13,21:30,2019-03-13,23:10,305.360682
490942,KLM,Mobile,BFM,Atlanta,FFC,2019-03-18,12:45,2019-03-18,20:25,1036.829838
440194,Delta,Miami,OPF,Atlanta,RYY,2019-03-14,09:46,2019-03-14,11:07,481.782781
390877,Delta,Atlanta,FTY,Lexington,LXN,2019-03-15,14:35,2019-03-15,16:20,1147.158865


### Asientos disponibles

In [44]:
a['asientos_disponibles'] = a.apply(lambda x: np.random.randint(0, 5), axis=1)
a.sample(5)

,aerolinea,origen,cod_origen,destino,cod_destino,fecha_salida,horario_salida,fecha_llegada,horario_llegada,precio,asientos_disponibles
325016,Alaska,Anchorage,LHD,Portland,PDX,2019-03-13,13:10,2019-03-13,14:40,1589.451569,2
429270,Delta,Fresno,FAT,Salt Lake City,SLC,2019-03-13,10:40,2019-03-13,12:00,868.634533,4
261612,American,Seattle,SEA,Miami,OPF,2019-03-10,16:58,2019-03-11,17:13,796.823231,3
130415,Aeromexico,Detroit,DET,Atlanta,ATL,2019-03-10,09:32,2019-03-10,12:23,1287.133404,2
130924,Aeromexico,Detroit,DTW,Atlanta,FFC,2019-03-13,06:35,2019-03-13,08:50,1377.368046,2


### Últimas modificaciones

In [ ]:
# Eliminamos columnas innecesarias
a = a.drop(columns=['cod_origen', 'cod_destino'])

In [47]:
# Pasamos a entero el precio (para mejorar la performance)
a['precio'] = a['precio'].astype(int)

In [48]:
a.sample(5)

,aerolinea,origen,destino,fecha_salida,horario_salida,fecha_llegada,horario_llegada,precio,asientos_disponibles
280218,Air France,Atlanta,Brunswick,2019-03-12,14:00,2019-03-12,15:43,740,0
460997,Norwegian,Oslo,Stavanger,2019-03-14,16:45,2019-03-14,18:10,780,3
95750,Air France,Atlanta,Denver,2019-03-10,11:15,2019-03-10,11:25,496,4
487077,United,Columbus,Denver,2019-03-21,14:35,2019-03-21,16:08,581,0
395848,Delta,Atlanta,Myrtle Beach,2019-03-15,21:40,2019-03-15,23:59,341,1


In [49]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371905 entries, 2 to 525849
Data columns (total 9 columns):
aerolinea               371905 non-null object
origen                  371905 non-null object
destino                 371905 non-null object
fecha_salida            371905 non-null object
horario_salida          371905 non-null object
fecha_llegada           371905 non-null object
horario_llegada         371905 non-null object
precio                  371905 non-null int32
asientos_disponibles    371905 non-null int64
dtypes: int32(1), int64(1), object(7)
memory usage: 27.0+ MB


## Exportación del CSV

In [50]:
a.to_csv('../csv/info_vuelos.csv', index = None, header = True)